In [1]:
import pandas as pd
import numpy as np
import random

random.seed(123)

The cell below imports the card values (7 decks) and strategies to be used for evaluating each hand.

In [2]:
cards = pd.read_csv('7_Decks_of_Cards.csv', sep=',',header=0)
double_strat = pd.read_csv('Double_Down_Strat.csv', sep=',',header=0)

split_strat = pd.read_csv('Split_Strat.csv', sep=',',header=0)
split_strat = split_strat.set_index('Split_Strat')

hit_strat = pd.read_csv('Hit_Strat.csv', sep=',',header=0)
hit_strat = hit_strat.set_index('Hit_Strat')

The cell below establishes the starting parameters of the simulation including # of replications.

In [3]:
# Hands will stop being played when max_hands_to_play variable is met, or if the money tracker runs to zero.
# Each hand will bet the default bet, unless otherwise dictated by strategy
max_hands_to_play = 100  # aka replications
initial_money = 1000
default_bet = 10

The main_advanced function below lays out the framework for the Blackjack Simulation using the most advanced strategy (split, doubledown, and basic hit/stay included). Also incorporating the # of games, budget, and bet parameters set above.

In [4]:
def main_advanced(initial_money = initial_money, replications = max_hands_to_play, default_bet = default_bet):
    
    # The money_tracker will show the total value of the initial funds as well as money won/lost
    money_tracker = initial_money
    
    # The dictionary below is for collecting the key performance indicators for each run of the simulation. 
    # These KPI's will be used to conduct the comparison of each Blackjack strategy.
    sim_metrics = {'strategy': ['Advanced'], 'initial_money': [initial_money], 'final_money': [222], 'games_played': [0], 'wins': [0], 'split_count': [0], 
                   'split_win': [0], 'split_winnings': [0], 'double_count': [0], 'double_win': [0], 'double_winnings': [0]}
    
    for i in range(replications):
        
        sim_metrics['games_played'][0] += 1
        
        tbet = default_bet
        tbet_split = 0
        
        player, dealer = start()

        player, dealer = Deal_Cards(player, dealer)
        
        # The following code checks if splitting the hand is allowed and then strategically advantageous
        
        play_split = evaluate_split(player, dealer)
        
        if play_split == True:
            sim_metrics['split_count'][0] += 1
            tbet_split = tbet
            split_start(player, dealer)
            
            double_down_split = evaluate_double_split(player, dealer)
            
            if double_down_split == True:
                sim_metrics['double_count'][0] += 1
                tbet_split += tbet
                player = split_hit(player)
            else:
                player = Player_Evaluate_Cards_Split(player, dealer)
                
        
        # The following code initiates either a double down strategy or a standard hit/stay strategy
        double_down = evaluate_double(player, dealer)
        
        if double_down == True:
            sim_metrics['double_count'][0] += 1
            tbet += tbet
            player = hit(player)
        else:
            player = Player_Evaluate_Cards(player, dealer)  # Standard hit/stay strategy
        
        dealer = Dealer_Evaluate_Cards(dealer)  # Dealer evaluates his cards based on standard rules
        
        
        if play_split == True:
            
            win_split = win_lose_split(player, dealer)
            
            if win_split == 1:
                sim_metrics['split_win'][0] += 1
                sim_metrics['split_winnings'][0] += tbet_split
                money_tracker += tbet_split
                
                if double_down_split == True:
                    sim_metrics['double_win'][0] += 1
                    sim_metrics['double_winnings'][0] += tbet_split
            
            elif win_split == 0.001:
                money_tracker = money_tracker
            
            else:
                sim_metrics['split_winnings'][0] -= tbet
                money_tracker -= tbet_split
            
                if double_down_split == True:
                    sim_metrics['double_winnings'][0] -= tbet_split
        
        win = win_lose(player, dealer)  # Comparison of hand results => 1 for player win, 0 if not
        sim_metrics['wins'][0] += win
        
        if win == 1:
            money_tracker += tbet
        elif win == 0.001:
            money_tracker = money_tracker
        else:
            money_tracker -= tbet
        
        if win == 1 and double_down == True:
            sim_metrics['double_win'][0] += 1
            sim_metrics['double_winnings'][0] += tbet
        elif win == 0 and double_down == True:
            sim_metrics['double_winnings'][0] -= tbet
        
        sim_metrics['final_money'][0] = money_tracker
        
        if money_tracker <= 0:
            return sim_metrics
    
    return sim_metrics

The main_split function below lays out the framework for the Blackjack Simulation using the split only strategy (doubledown is excluded). Also incorporating the # of games, budget, and bet parameters set above.

In [5]:
def main_split(initial_money = initial_money, replications = max_hands_to_play, default_bet = default_bet):
    
    # The money_tracker will show the total value of the initial funds as well as money won/lost
    money_tracker = initial_money
    
    sim_metrics = {'strategy': ['Split'], 'initial_money': [initial_money], 'final_money': [222], 'games_played': [0], 'wins': [0], 'split_count': [0], 
                   'split_win': [0], 'split_winnings': [0], 'double_count': [0], 'double_win': [0], 'double_winnings': [0]}
    
    for i in range(replications):
        
        sim_metrics['games_played'][0] += 1
        
        tbet = default_bet
        tbet_split = 0
        
        player, dealer = start()

        player, dealer = Deal_Cards(player, dealer)
        
        # The following code checks if splitting the hand is allowed and then strategically advantageous
        play_split = evaluate_split(player, dealer)
        
        if play_split == True:
            sim_metrics['split_count'][0] += 1
            tbet_split = tbet
            split_start(player, dealer)
            
            player = Player_Evaluate_Cards_Split(player, dealer)
            
        
        player = Player_Evaluate_Cards(player, dealer)  # Standard hit/stay strategy
        
        dealer = Dealer_Evaluate_Cards(dealer)  # Dealer evaluates his cards based on standard rules
        
        if play_split == True:
            
            win_split = win_lose_split(player, dealer)
            
            if win_split == 1:
                sim_metrics['split_win'][0] += 1
                sim_metrics['split_winnings'][0] += tbet_split
                money_tracker += tbet_split
            
            elif win_split == 0.001:
                money_tracker = money_tracker
            
            else:
                sim_metrics['split_winnings'][0] -= tbet
                money_tracker -= tbet_split
        
        win = win_lose(player, dealer)  # Comparison of hand results => 1 for player win, 0 if not
        sim_metrics['wins'][0] += win
        
        if win == 1:
            money_tracker += tbet
        elif win == 0.001:
            money_tracker = money_tracker
        else:
            money_tracker -= tbet
        
        sim_metrics['final_money'][0] = money_tracker
        
        if money_tracker <= 0:
            return sim_metrics
    
    return sim_metrics

The main_double function below lays out the framework for the Blackjack Simulation using the doubledown only strategy (split is excluded). Also incorporating the # of games, budget, and bet parameters set above.

In [6]:
def main_double(initial_money = initial_money, replications = max_hands_to_play, default_bet = default_bet):
    
    # The money_tracker will show the total value of the initial funds as well as money won/lost
    money_tracker = initial_money
    
    sim_metrics = {'strategy': ['Double'], 'initial_money': [initial_money], 'final_money': [222], 'games_played': [0], 'wins': [0], 'split_count': [0], 
                   'split_win': [0], 'split_winnings': [0], 'double_count': [0], 'double_win': [0], 'double_winnings': [0]}
    
    for i in range(replications):
        
        sim_metrics['games_played'][0] += 1
        
        tbet = default_bet
        tbet_split = 0
        
        player, dealer = start()

        player, dealer = Deal_Cards(player, dealer)
        
        # The following code initiates either a double down strategy or a standard hit/stay strategy
        double_down = evaluate_double(player, dealer)
        
        if double_down == True:
            sim_metrics['double_count'][0] += 1
            tbet += tbet
            player = hit(player)
        else:
            player = Player_Evaluate_Cards(player, dealer)  # Standard hit/stay strategy
        
        dealer = Dealer_Evaluate_Cards(dealer)  # Dealer evaluates his cards based on standard rules
        
        win = win_lose(player, dealer)  # Comparison of hand results => 1 for player win, 0 if not
        sim_metrics['wins'][0] += win
        
        if win == 1:
            money_tracker += tbet
        elif win == 0.001:
            money_tracker = money_tracker
        else:
            money_tracker -= tbet
        
        if win == 1 and double_down == True:
            sim_metrics['double_win'][0] += 1
            sim_metrics['double_winnings'][0] += tbet
        elif win == 0 and double_down == True:
            sim_metrics['double_winnings'][0] -= tbet
        
        sim_metrics['final_money'][0] = money_tracker
        
        if money_tracker <= 0:
            return sim_metrics
    
    return sim_metrics

The main_basic function below lays out the framework for the Blackjack Simulation using the most basic hit/stay strategy (split and doubledown are excluded). Also incorporating the # of games, budget, and bet parameters set above.

In [7]:
def main_basic(initial_money = initial_money, replications = max_hands_to_play, default_bet = default_bet):
    
    # The money_tracker will show the total value of the initial funds as well as money won/lost
    money_tracker = initial_money
    
    sim_metrics = {'strategy': ['Basic'], 'initial_money': [initial_money], 'final_money': [222], 'games_played': [0], 'wins': [0], 'split_count': [0], 
                   'split_win': [0], 'split_winnings': [0], 'double_count': [0], 'double_win': [0], 'double_winnings': [0]}
    
    for i in range(replications):
        
        sim_metrics['games_played'][0] += 1
        
        tbet = default_bet
        tbet_split = 0
        
        player, dealer = start()

        player, dealer = Deal_Cards(player, dealer)
        
        player = Player_Evaluate_Cards(player, dealer)  # Standard hit/stay strategy
        
        dealer = Dealer_Evaluate_Cards(dealer)  # Dealer evaluates his cards based on standard rules
        
        win = win_lose(player, dealer)  # Comparison of hand results => 1 for player win, 0 if not
        sim_metrics['wins'][0] += win
        
        if win == 1:
            money_tracker += tbet
        elif win == 0.001:
            money_tracker = money_tracker
        else:
            money_tracker -= tbet
        
        sim_metrics['final_money'][0] = money_tracker
        
        if money_tracker <= 0:
            return sim_metrics
    
    return sim_metrics

The function defined below establishes the structure of the game variables (empty hands) and (re)starts the simulation for replication.

In [8]:
#Initial values to be used as a starting point for each game played
def start():

    empty_hand = {'Card': [0, 0, 0, 0, 0, 0, 0], 'Value': [0, 0, 0, 0, 0, 0, 0], 
                  'Split': [0, 0, 0, 0, 0, 0, 0], 'Split_Value': [0, 0, 0, 0, 0, 0, 0]}

    #Setting up the player's hand to be filled in over the course of gameplay
    players_hand = pd.DataFrame(data=empty_hand)

    #Setting up the dealer's hand to be filled in over the course of gameplay
    dealers_hand = pd.DataFrame(data=empty_hand)
    
    return players_hand, dealers_hand

The Deal_Cards sets up initial values to be evaluated.

In [9]:
def Deal_Cards(players_hand, dealers_hand):
    for i in range(2):
        tCard = cards.iloc[random.randint(1,363), 1:3]
        
        players_hand.iloc[i, 0] = tCard.iloc[0]
        players_hand.iloc[i, 1] = tCard.iloc[1]
        
        tCard = cards.iloc[random.randint(1,363), 1:3]
        
        dealers_hand.iloc[i, 0] = tCard.iloc[0]
        dealers_hand.iloc[i, 1] = tCard.iloc[1]
    
    return players_hand, dealers_hand

The hit() function defined below adds a card to the player or dealers hand when strategy dictates. 

In [10]:
def hit(hand):
    
    tCard = cards.iloc[random.randint(1,363), 1:3]
    
    for i in range(1,7):
        if hand.iloc[i, 1] == 0:
            hand.iloc[i, 0] = tCard.iloc[0]
            hand.iloc[i, 1] = tCard.iloc[1]
            return hand

The split_hit() function defined below adds a card to the players 2nd hand when the split strategy is active. 

In [11]:
def split_hit(hand):
    
    tCard = cards.iloc[random.randint(1,363), 1:3]
    
    for i in range(1,7):
        if hand.iloc[i, 3] == 0:
            hand.iloc[i, 2] = tCard.iloc[0]
            hand.iloc[i, 3] = tCard.iloc[1]
            return hand

The Player_Evaluate_Cards function determines whether the player will hit or stay based on the original hand and the hit_strat dataframe. If there's an Ace in a hand, and that player busts, the Ace value converts from 11 to one.

In [12]:
def Player_Evaluate_Cards(players_hand, dealers_hand, hit_strat = hit_strat):
    
    player_hand_value = players_hand.iloc[:,1].sum()
    
    dealer_show = str(dealers_hand.iloc[0, 1])
    
    while player_hand_value <= 7:
        players_hand = hit(players_hand)
        player_hand_value = players_hand.iloc[:,1].sum()
    
    while player_hand_value <= 17 and hit_strat.loc[player_hand_value, dealer_show] == True:
        players_hand = hit(players_hand)
        player_hand_value = players_hand.iloc[:,1].sum()
        
    if player_hand_value > 21 and 'Ace' in players_hand.iloc[0].unique():
        loca = players_hand.loc[players_hand['Card'] == 'Ace', 'Value']
        ind = list(loca.index)
        players_hand.iloc[ind[0], 1] = 1
        player_hand_value = players_hand.iloc[:,1].sum()
    
    while player_hand_value <= 17 and hit_strat.loc[player_hand_value, dealer_show] == True:
        players_hand = hit(players_hand)
        player_hand_value = players_hand.iloc[:,1].sum()
    
    return players_hand


The Player_Evaluate_Cards_Split function determines whether the player will hit or stay based on the split hand and the hit_strat dataframe. If there's an Ace in a hand, and that player busts, the Ace value converts from 11 to one.

In [13]:
def Player_Evaluate_Cards_Split(players_hand, dealers_hand, hit_strat = hit_strat):
    
    player_hand_value = players_hand.iloc[:,3].sum()
    
    dealer_show = str(dealers_hand.iloc[0, 1])
    
    while player_hand_value <= 7:
        players_hand = split_hit(players_hand)
        player_hand_value = players_hand.iloc[:,3].sum()
    
    while player_hand_value <= 17 and hit_strat.loc[player_hand_value, dealer_show] == True:
        players_hand = split_hit(players_hand)
        player_hand_value = players_hand.iloc[:,3].sum()
        
    if player_hand_value > 21 and 'Ace' in players_hand.iloc[2].unique():
        loca = players_hand.loc[players_hand['Split'] == 'Ace', 'Split_Value']
        ind = list(loca.index)
        players_hand.iloc[ind[0], 3] = 1
        player_hand_value = players_hand.iloc[:,3].sum()
    
    while player_hand_value <= 17 and hit_strat.loc[player_hand_value, dealer_show] == True:
        players_hand = split_hit(players_hand)
        player_hand_value = players_hand.iloc[:,3].sum()
    
    return players_hand

The evaluate_split function determines whether or not the player will split their original cards into two separate hands based on the split_strat dataframe.

In [14]:
def evaluate_split(players_hand, dealers_hand, split_strat = split_strat):
    
    dealer_show = str(dealers_hand.iloc[0, 1])
    player_card = str(players_hand.iloc[0, 0])
    
    #print(dealer_show)
    #print(player_card)
    
    if players_hand.iloc[0, 0] != players_hand.iloc[1, 0]:
        return False

    elif split_strat.loc[player_card, dealer_show] == True:
        return True
    
    else:
        return False

The split_start function adjusts the structure of the game to accomodate a split hand. Triggered when evaluate_split comes back True.

In [15]:
def split_start(players_hand, dealers_hand, split_strat = split_strat):
    players_hand = {'Card': [players_hand.iloc[0,0], 0, 0, 0, 0, 0, 0], 'Value': [players_hand.iloc[0,1], 0, 0, 0, 0, 0, 0], 
                  'Split': [players_hand.iloc[1,0], 0, 0, 0, 0, 0, 0], 'Split_Value': [players_hand.iloc[1,1], 0, 0, 0, 0, 0, 0]}
    
    players_hand = pd.DataFrame(data=players_hand)
    
    players_hand = hit(players_hand)
    players_hand = split_hit(players_hand)
    
    return players_hand

The evaluate_double function determines whether or not the player will double down on the bet from the original hand based on the double_strat dataframe.

In [16]:
def evaluate_double(players_hand, dealers_hand, double_strat = double_strat):
    
    card1 = players_hand.iloc[0,1]
    card2 = players_hand.iloc[1,1]
    
    dealer_card = str(dealers_hand.iloc[0, 1])
    
    
    player_card1_1 = card1 in double_strat.iloc[:,0].unique()
    player_card1_2 = card1 in double_strat.iloc[:,1].unique()
    
    player_card2_1 = card2 in double_strat.iloc[:,0].unique()
    player_card2_2 = card2 in double_strat.iloc[:,1].unique()
    
    
    if player_card1_1 == False and player_card1_2 == False:
        #print(1)
        return False
    
    elif player_card2_1 == False and player_card2_2 == False:
        #print(2)
        return False
    
    elif player_card1_1 == True and player_card2_2 == True:
        result = double_strat.loc[double_strat['card1'] == card1]
        
        if card2 in result['card2'].unique():
            result = result[result['card2'] == card2]
            #print(3)
            return result.loc[:, dealer_card].values
        
        else:
            return False
    
    elif player_card1_2 == True and player_card2_1 == True:
        result = double_strat.loc[double_strat['card1'] == card2]
            
        if card1 in result['card2'].unique():  
            result = result[result['card2'] == card1]
            #print(4)
            return result.loc[:, dealer_card].values
        
        else:
            return False
    
    else:
        #print(5)
        return False
    

The evaluate_double_split function determines whether or not the player will double down on the bet from the split hand based on the double_strat dataframe.

In [17]:
def evaluate_double_split(players_hand, dealers_hand, double_strat = double_strat):
    
    card1 = players_hand.iloc[0,3]
    card2 = players_hand.iloc[1,3]
    
    dealer_card = str(dealers_hand.iloc[0, 1])
    
    
    player_card1_1 = card1 in double_strat.iloc[:,0].unique()
    player_card1_2 = card1 in double_strat.iloc[:,1].unique()
    
    player_card2_1 = card2 in double_strat.iloc[:,0].unique()
    player_card2_2 = card2 in double_strat.iloc[:,1].unique()
    
    if player_card1_1 == False and player_card1_2 == False:
        return False
    
    elif player_card2_1 == False and player_card2_2 == False:
        return False
    
    elif player_card1_1 == True and player_card2_2 == True:
        result = double_strat.loc[double_strat['card1'] == card1]
        
        if card2 in result['card2'].unique():
            result = result[result['card2'] == card2]
            return result.loc[:, dealer_card].values
        
        else:
            return False
    
    elif player_card1_2 == True and player_card2_1 == True:
        result = double_strat.loc[double_strat['card1'] == card2]
        
        if card1 in result['card2'].unique():
            result = result[result['card2'] == card1]
            return result.loc[:, dealer_card].values
        
        else:
            return False
    
    else:
        return False

The Dealer_Evaluate_Cards function determines the next action to be taken by the dealer (Hit or Stay).

In [18]:
def Dealer_Evaluate_Cards(dealers_hand):
    dealer_hand_value = dealers_hand.iloc[:,1].sum()
    
    while dealer_hand_value <= 16:
        hit(dealers_hand)
        dealer_hand_value = dealers_hand.iloc[:,1].sum()
        
        if dealer_hand_value > 21 and 'Ace' in dealers_hand.iloc[0].unique():
            loca = dealers_hand.loc[dealers_hand['Card'] == 'Ace', 'Value']
            ind = list(loca.index)
            dealers_hand.iloc[ind[0], 1] = 1
            dealer_hand_value = dealers_hand.iloc[:,1].sum()
    
    return dealers_hand
    

The win_lose function evaluates the final value of the player and dealer's hands to see who has the highest value without busting (going over 21).

In [19]:
def win_lose(players_hand, dealers_hand):
    
    player_bust = False
    dealer_bust = False
    
    player_hand_value = players_hand.iloc[:,1].sum()
    dealer_hand_value = dealers_hand.iloc[:,1].sum()
    
    
    if player_hand_value >= 22:
        player_bust = True
    
    if dealer_hand_value >= 22:
        dealer_bust = True
        
    
    if player_bust == True:
        return 0
    
    elif player_bust == False and dealer_bust == True:
        return 1
    
    elif player_bust == False and dealer_bust == False:
        if player_hand_value < dealer_hand_value:
            return 0
        elif player_hand_value == dealer_hand_value:
            return 0.001
        else:
            return 1
    else:
        return 1
    

The win_lose_split function evaluates the final value of the player's split hand against the dealer's hand to see who has the highest value without busting (going over 21).

In [20]:
def win_lose_split(players_hand, dealers_hand):
    
    player_bust = False
    dealer_bust = False
    
    player_hand_value = players_hand.iloc[:,3].sum()
    dealer_hand_value = dealers_hand.iloc[:,1].sum()
    
    
    if player_hand_value >= 22:
        player_bust = True
    
    if dealer_hand_value >= 22:
        dealer_bust = True
        
    
    if player_bust == True:
        return 0
    
    elif player_bust == False and dealer_bust == True:
        return 1
    
    elif player_bust == False and dealer_bust == False:
        if player_hand_value < dealer_hand_value:
            return 0
        elif player_hand_value == dealer_hand_value:
            return 0.001
        else:
            return 1
    else:
        return 1

The cell below generates simulation data for the 4 strategies: Advanced, Double, Split, and Basic. All data is collected into a single dataframe.

In [21]:
Final_Data_test4 = pd.DataFrame(data = {'strategy': [], 'initial_money': [], 'final_money': [], 'games_played': [], 'wins': [], 
                                      'split_count': [], 'split_win': [], 'split_winnings': [], 'double_count': [], 
                                      'double_win': [], 'double_winnings': []})

for i in range(100):
    Final_Data_test4 = Final_Data_test4.append(pd.DataFrame(data = main_advanced()), sort=False)
    Final_Data_test4 = Final_Data_test4.append(pd.DataFrame(data = main_split()), sort=False)
    Final_Data_test4 = Final_Data_test4.append(pd.DataFrame(data = main_double()), sort=False)
    Final_Data_test4 = Final_Data_test4.append(pd.DataFrame(data = main_basic()), sort=False)
    
Final_Data_test4.head()

    

,strategy,initial_money,final_money,games_played,wins,split_count,split_win,split_winnings,double_count,double_win,double_winnings
0,Advanced,1000.0,800.0,100.0,32.016,1.0,0.0,-10.0,9.0,5.0,20.0
0,Split,1000.0,1010.0,100.0,46.009,2.0,1.0,0.0,0.0,0.0,0.0
0,Double,1000.0,810.0,100.0,38.008,0.0,0.0,0.0,12.0,4.0,-60.0
0,Basic,1000.0,890.0,100.0,40.009,0.0,0.0,0.0,0.0,0.0,0.0
0,Advanced,1000.0,980.0,100.0,43.007,1.0,1.0,10.0,6.0,5.0,80.0


In [22]:
Final_Data_test4.to_csv('sim_output1.csv', index = False)

In [ ]:
hist